In [1]:
import time
# pour la gestion du GPU, j'utilise thinc.api finalement au lieu de spacy

# pour un ordi avec GPU :

import GPUtil
import torch
!set CUDA_PATH="/usr/local/cuda-12"
GPUtil.getAvailable()
use_cuda = torch.cuda.is_available()
print("is cuda available? ", use_cuda)

is cuda available?  True


In [2]:
import pandas as pd
import spacy

In [3]:
def print_full(df):
    pd.set_option('display.max_colwidth', 2000)
    # pd.set_option('display.width', 2000)

    print(df)
    # pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 2000)
# pd.set_option('display.float_format', '{:20,.2f}'.format)
# pd.set_option('display.max_colwidth', None)
# print(x)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.width')
# pd.reset_option('display.float_format')
# pd.reset_option('display.max_colwidth')

In [4]:

# vérifier l'état des données

def check_data(df_to_check):

    if df_to_check.isnull().values.any() :
        print("y a t il des nan? ->",df_to_check.isnull().values.any())
        print("combien y a t il de nan? ->",df_to_check.isnull().values.sum())
        print("où sont les null? ->\n",df_to_check.isnull().sum())
        print(df_to_check[df_to_check.isnull().T.any()])
    else :
        print("il n'y a pas de NaN")

    print("quels types pour chaque colonne?\n")    
    for column in df_to_check.columns:
        print(pd.api.types.infer_dtype(df_to_check[column]))
    print("y a-t-il des types mixés dans les données?\n")
    print(df_to_check[column][df_to_check[column].apply(lambda x: isinstance(x, type))])
    print(df_to_check._is_mixed_type)
    print(df_to_check.dtypes.nunique()>1)

In [5]:
from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu, use_pytorch_for_gpu_memory, set_active_gpu
set_gpu_allocator("pytorch")
require_gpu() 
set_active_gpu(0)
if prefer_gpu():
    print("Using GPU!")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    use_pytorch_for_gpu_memory()

    print("GPU Usage")
    GPUtil.showUtilization()


Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 21% |  6% |


In [6]:
# Check if spacy is using GPU

gpu_usage = spacy.prefer_gpu()
if gpu_usage:
    print("spaCy is using GPU.")
else:
    print("spaCy is using CPU.")

spaCy is using GPU.


In [5]:
df = pd.read_csv("df_CC_cleaned.csv", sep = ",", encoding = "utf-8", dtype= str)
check_data(df)

In [7]:

def lemmatize(doc, batch_size, stopwords = False):
    
    '''
    - split the dataframe into batches
    - take each batche and apply choosed nlp model from spacy
    - retrieve lemmes texts and corresponding postags
    
    Input : doc = a text column from a dataframe, batch_size = an int, stopwords = boolean 
    stopwords is designed to choose if we want to keep only NOUN, ADJ, ADV, VERB lemmes and delete other

    Output :  one df with lemmas texts and one df with correpsonding postags 
    '''

    nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
    nlp.max_length = 3000000 #this is possible as we're not using parser or ner.

    t0 = time.time()
    
    # count the number of batches
    len_data = doc.shape[0]
    print("batch_size : ", batch_size)
    nb_batch = int(len_data/batch_size)+ (len_data % batch_size > 0)
    print("nb_batch : ", nb_batch)
    # size of last bacth
    size_last_batch = len_data%batch_size
    print("size_last_batch : ", size_last_batch)
    first_idx = 0
    last_idx = 0
        
    final_lemmas_df = pd.DataFrame()
    final_postag_df = pd.DataFrame()


    # creating one batch (a slice of the df)
    for batch_nb in range(nb_batch) :
        t1 = time.time()
        lemma_text_list = []
        lemma_pos_list = []
        print(f"starting batch n° {batch_nb} , on {nb_batch} batches")
        # range() commence à 0 et exclut nb_batch : donc on utilise nb_batch -1 pout détecter le dernier batch
        # si c'est le dernier batch, on ajoute la taille du batch final au lieu de la taille prévue
        if batch_nb == nb_batch -1 :
            first_idx = last_idx
            last_idx += size_last_batch
        else :
            first_idx = last_idx
            last_idx += batch_size
        print(f"first_index : {first_idx} and last_idx : {last_idx}")
        batch = doc[first_idx : last_idx]
        
        # spacy tools go through the batch
        for line in nlp.pipe(batch):
            if stopwords is True:
                lemma_pos_list.append([x.pos_ for x in line if x.pos_ in {"VERB", "ADJ", "ADV", "NOUN"}])
                lemma_text_list.append([x.lemma_ for x in line if x.pos_ in {"VERB", "ADJ", "ADV","NOUN"}])

            else :
                lemma_pos_list.append([x.pos_ for x in line])
                lemma_text_list.append([x.lemma_ for x in line])

        print("GPU Usage")
        GPUtil.showUtilization()

        # writing the output lemmas and postag in one file each
        # n contributions per file, n the batch_size

        df_lemma = pd.DataFrame([lemma_text_list])
        df_postag = pd.DataFrame([lemma_pos_list])


        final_lemmas_df = pd.concat([final_lemmas_df,df_lemma.T])
        final_postag_df = pd.concat([final_postag_df,df_postag.T])

        # deleting the lists for clearing, to force python garbage collector to *actually* collect
        del df_lemma
        del df_postag

        del lemma_text_list
        del lemma_pos_list
    
        t2 = time.time()
        print(f"end of batch n°: {batch_nb}, time: {t2-t1}")
        
    t3 = time.time()
    print("Total time: {}".format(t3-t0))
    return(final_lemmas_df, final_postag_df)


spaCy is using GPU.


In [ ]:
# lemmatize and postag on clean text
final_lemmas_df, final_postag_df = lemmatize(df["clean"], batch_size= 256, stopwords = False)

In [9]:
# adding a column to the df
# renumber the index column (not accurate bc origin from each concatenation)

forconcat_lemmas_df = final_lemmas_df.reset_index()
forconcat_postag_df = final_postag_df.reset_index()

# drop the secoond index column

forconcat_lemmas_df = forconcat_lemmas_df.drop(labels = "index", axis=1)
forconcat_postag_df = forconcat_postag_df.drop(labels = "index", axis=1)

# name the columns 

forconcat_lemmas_df.rename(columns = {0:"lemmas"}, inplace=True)
forconcat_postag_df.rename(columns = {0:"postags"}, inplace=True)

# concatenate the columns to the side of the original csv

df = pd.concat([df,forconcat_lemmas_df], axis = 1)
df = pd.concat([df,forconcat_postag_df], axis = 1)

In [10]:
# exporter le dataframe dans un csv

df.to_csv("df_CC_lemmas_postag.csv", index = False)

In [ ]:
# lemmatize and postagging on only verb, noun, adj, adv
model_lemmas_df, model_postag_df = lemmatize(df["clean"], batch_size= 256, stopwords = True)

In [12]:
# renumber the index column (not accurate bc origin from each concatenation)

stopwords_lemmas_df = model_lemmas_df.reset_index()
stopwords_postag_df = model_postag_df.reset_index()

# drop the secoond index column

stopwords_lemmas_df = stopwords_lemmas_df.drop(labels = "index", axis=1)
stopwords_postag_df = stopwords_postag_df.drop(labels = "index", axis=1)

# add these stopwords_lemmas_df and  stopwords_postag_df to the global df
df["lemmas_only_VERB_ADJ_ADV_NOUN"] = stopwords_lemmas_df
df["postags_only_VERB_ADJ_ADV_NOUN"] = stopwords_postag_df


In [14]:
# exporter le dataframe dans un csv

# df.to_csv("df_CC_lemmas_postag.csv", index = False)

In [7]:
df = pd.read_csv("df_CC_lemmas_postag.csv", sep = ",", encoding = "utf-8", dtype= str)


In [8]:
check_data(df)

il n'y a pas de NaN
quels types pour chaque colonne?

string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
y a-t-il des types mixés dans les données?

Series([], Name: postags_only_VERB_ADJ_ADV_NOUN, dtype: object)
False
False


In [ ]:
# delete the stopwords, using the spacy lexicon

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]
final_stopwords_set = set(final_stopwords_list)

def remove_stopwords(doc):
    output = []

    for sentence_list in doc :
        sentence_output = []
        print('in remove sentence : ', sentence_list)
        for word in sentence_list :
            print("in remove   ", word)
        if word not in final_stopwords_set :
            sentence_output.append(word)
        output.append(sentence_output)
        
    return(output)


def batching_stopwords(doc_as_list, batch_size) : 
    """
    applique remove_dots_and_spaces() sur chaque cellule de texte du dataframe

    Input : une colonne de textes sélectionnée dans un dataframe , une taille de batch
    Output : un dataframe avec toutes les string modifiées 
    """

    # count the number of batches
    len_data = len(doc_as_list)
    batch_total = int(len_data/batch_size)+ (len_data % batch_size > 0)

    # size of last bacth
    size_last_batch = len_data%batch_size
    first_idx = 0
    last_idx = 0

    output_df = pd.DataFrame()
    for batch in range(batch_total) :

        #  detect last batch
        if batch == batch_total -1 :
            first_idx = last_idx
            last_idx += size_last_batch
        # 
        else :
            first_idx = last_idx
            last_idx += batch_size
        print(first_idx, last_idx)
        for text_cell in doc_as_list[first_idx:last_idx] :
            print("text_cell :", text_cell)
            output = remove_stopwords(text_cell)
            print("text_cell_clean :", text_cell)

        # concatene this batch list of text with the complete dataframe 
        output_df = pd.concat([output_df, pd.DataFrame({"name" : output})], ignore_index= True )
        del output

    return output_df



# doc = df.lemmas.tolist()
# all_lemmas = batching_stopwords(doc, 256)
# df["all_lemmas_without_stopwords_set"] = all_lemmas

# doc = df.lemmas_only_VERB_ADJ_ADV_NOUN.tolist()
# some_lemmas = batching_stopwords(doc, 256)
# df["VERB_ADJ_ADV_NOUN_without_stopwords"] = some_lemmas
# doc = df.clean.tolist()
# some_clean_text = batching_stopwords(doc, 256)
# df["clean_text_without_stopwords"] = some_clean_text
df_test = df[0:1000]
doc = df_test.clean.tolist()
some_clean_text = remove_stopwords(doc)
df_test["clean_text_without_stopwords"] = some_clean_text


In [12]:
# exporter le dataframe dans un csv

# df.to_csv("df_CC_lemmas_postag.csv", index = False)